In [ ]:
import pandas
import numpy
import matplotlib.pyplot as plt
import matplotlib
import sys
import os
import pyarrow.parquet as pq
import folium
import numpy as np

In [ ]:
path = "data/"
# ouvrir les fichiers
incidents = pandas.read_csv(path+'donnees_incidents.csv', sep=';')
souterrain_2023 = pandas.read_csv(path+'souterrain_2023.csv', sep=';')
souterrain_troncons = pandas.read_csv(path+'souterrain_tronçons.csv', sep=';')
icu = pandas.read_csv(path+'icu.csv', sep=';')
temperatures = pq.read_table(path+'temperatures.parquet').to_pandas()
humidite = pq.read_table(path+'humidite.parquet').to_pandas()
nuages = pq.read_table(path+'nuages.parquet').to_pandas()

# plotly show number of incidents per month [january to december]
incidents_per_month = incidents.groupby('Mois (nom)').size()
# order months by chronological order
incidents_per_month = incidents_per_month.reindex(['janv', 'févr', 'mars', 'avr', 'mai', 'juin', 'juil', 'août', 'sept', 'oct', 'nov', 'déc'])
incidents_per_month.plot.bar()
plt.title('Number of incidents per month')
plt.xlabel('Month')
plt.ylabel('Number of incidents')
plt.show()

In [ ]:
# show incidents on a map
# create a map
m = folium.Map(location=[45.75, 4.85], zoom_start=12)
# add incidents
for index, row in incidents.iterrows():
    try :
        folium.Marker([row['new_y'], row['new_x']], popup=row['Cause de l\'interruption']).add_to(m)
    except :
        print("error: ", row['new_x'], row['new_y'])
        pass
# add underground cables
for index, row in souterrain_2023.iterrows():
    try : 
        folium.PolyLine([[row['y0'], row['x0']], [row['y1'], row['x1']]], color="red", weight=2.5, opacity=1).add_to(m)
    except :
        print("error: ", row['x0'], row['y0'], row['x1'], row['y1'])
        pass


In [ ]:
print(temperatures)
# date_valid     valeur  longitude  latitude
# 0       2014-01-01 00:00:00   6.712457        0.1      44.0
# 1       2014-01-01 01:00:00   6.176080        0.1      44.0

In [ ]:
# xgboost model to predict if there will be an incident or not on a given day (taking into account temperature)

# create a dataframe with the number of incidents per day (Année-Mois |  Semaine |  jour) and fill with 0 when there is no incident on a given day
incidents_per_day = incidents.groupby(['Année-Mois', 'Semaine', 'jour']).size()

incidents_per_day = incidents_per_day.reset_index()
incidents_per_day = incidents_per_day.rename(columns={0: 'Value'})
incidents_per_day['Année-Mois-jour'] = incidents_per_day['Année-Mois'] + '-' + incidents_per_day['jour'].astype(str)
incidents_per_day = incidents_per_day.drop(columns=['Année-Mois', 'Semaine', 'jour'])
incidents_per_day = incidents_per_day.set_index('Année-Mois-jour')

# add missing days with 0 value
start_date = incidents_per_day.index[0]
end_date = incidents_per_day.index[-1]
idx = pandas.date_range(start_date, end_date)
incidents_per_day.index = pandas.DatetimeIndex(incidents_per_day.index)
incidents_per_day = incidents_per_day.reindex(idx, fill_value=0)
incidents_per_day = incidents_per_day.reset_index()
incidents_per_day = incidents_per_day.rename(columns={'index': 'Année-Mois-jour'})
incidents_per_day['Année-Mois-jour'] = incidents_per_day['Année-Mois-jour'].astype(str)
incidents_per_day = incidents_per_day.set_index('Année-Mois-jour')
incidents_per_day = incidents_per_day.sort_index()
incidents_per_day.reset_index(inplace=True)

# create a dataframe with the temperature per day
temperatures_per_day = temperatures.groupby('date_valid').mean()
temperatures_per_day = temperatures_per_day.drop(columns=['longitude', 'latitude'])
temperatures_per_day.index = pandas.to_datetime(temperatures_per_day.index)
temperatures_per_day = temperatures_per_day.resample('D').mean()
temperatures_per_day = temperatures_per_day.dropna()
temperatures_per_day.reset_index(inplace=True)


In [ ]:
# incidents_per_day["seuil"] = np.where(incidents_per_day>1, 2, np.where(incidents_per_day>0, 1, 0))
# plt.figure(figsize=(15,5))
# plt.plot(incidents_per_day["seuil"])
# plt.xticks(range(0, len(incidents_per_day["seuil"]), 365),rotation=90)
# plt.title('Number of incidents per day')
# plt.xlabel('Date')
# plt.ylabel('Number of incidents')
# plt.show()

In [ ]:
# plt.figure(figsize=(15,5))
# plt.plot(incidents_per_day)
# plt.xticks(range(0, len(incidents_per_day.index), 365),rotation=90)
# plt.title('Number of incidents per day')
# plt.xlabel('Date')
# plt.ylabel('Number of incidents')
# plt.show()

In [ ]:
#show histogram of accident

incidents_per_day.hist(bins= range(10), figsize=(15,5))
plt.title('Histogram of number of incidents per day')
plt.xlabel('Number of incidents')
plt.xticks(range(10))
plt.ylabel('Number of days')
plt.show()

In [ ]:
import numpy as np
# plot mean temperature per day
plt.figure(figsize=(15,5))
plt.plot(temperatures_per_day)
plt.title('Mean temperature per day')
plt.xlabel('Date')
plt.ylabel('Mean temperature')
plt.show()

In [ ]:
humidite.rename(columns={"date_valid": 'Année-Mois-jour',"valeur": 'humidite'}, inplace=True)
nuages.rename(columns={"date_valid": 'Année-Mois-jour',"valeur": 'nuages'}, inplace=True)
# remove the H:MM:SS from the date
humidite['Année-Mois-jour'] = humidite['Année-Mois-jour'].str.split(' ').str[0].astype(str)
nuages['Année-Mois-jour'] = nuages['Année-Mois-jour'].str.split(' ').str[0].astype(str)


In [ ]:
humidite_per_day = humidite.groupby('Année-Mois-jour').mean()
nuages_per_day = nuages.groupby('Année-Mois-jour').mean()

In [ ]:
import pandas as pd
# Prepare dataframes so that column names match 
temperatures_per_day.rename(columns={"date_valid": 'Année-Mois-jour', 'valeur': 'temperature'}, inplace=True)
incidents_per_day.rename(columns={"Value": 'nb_incidents'}, inplace=True)

# Convert date to string for the merge
temperatures_per_day['Année-Mois-jour'] = temperatures_per_day['Année-Mois-jour'].astype(str)
# Merge the four dataframes on date column
incidents_and_temperatures_per_day = pd.merge(incidents_per_day, temperatures_per_day, on='Année-Mois-jour', how='left')
incidents_and_temperatures_per_day = pd.merge(incidents_and_temperatures_per_day, humidite_per_day, on='Année-Mois-jour', how='left')
incidents_and_temperatures_per_day = pd.merge(incidents_and_temperatures_per_day, nuages_per_day, on='Année-Mois-jour', how='left')

incidents_and_temperatures_per_day.drop(columns=['latitude_y', 'longitude_y'], inplace=True)
incidents_and_temperatures_per_day.rename(columns={"latitude_x": 'latitude', 'longitude_x': 'longitude'}, inplace=True)
incidents_and_temperatures_per_day.head()